In [1]:
from olfactorybulb.database import *
import os,sys
from neuronunit.tests.olfactory_bulb.publications import *
from neuronunit.tests.olfactory_bulb.tests import *
from neuronunit.models.neuron_cell import NeuronCellModel
from sciunit.suites import TestSuite
from pandas import DataFrame
import quantities as pq
from neuronunit.tests.olfactory_bulb.utilities import cache
from linetimer import CodeTimer
import string

/home/justas/anaconda2/envs/p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Could not load NEURONBackend


In [2]:
cell_type = 'mc'

# Load tests
measurements = Measurement\
    .select(Measurement)\
    .join(Property)\
    .switch(Measurement)\
    .join(Source)\
    .where((Measurement.property.type == "Electrophysiology") & (Measurement.property.id.startswith(cell_type+'_')))\
    .order_by(Measurement.property.id)

tests = []

for m in measurements:
    t = str(m.property.test_class_generic)
    pub = str(m.source.publication_class).strip()
    class_name = t+pub
    
    globals()[class_name] = type(class_name, 
                                 (eval(pub), eval(t)), 
                                 {})
    
    print('creating specific test class: ' + class_name)
    
    tests.append(eval(class_name)(observation={"mean": m.mean * eval(m.property.units), "std": m.std * eval(m.property.units), "n": m.n}))
    
    
# Load model classes
model_classes = list(CellModel\
                    .select(CellModel)\
                    .where(CellModel.cell_type == cell_type.upper())             
                     )
for i, m in enumerate(model_classes):
    nmsp = string.join(m.isolated_model_class.split('.')[:-1],'.')
    cls = m.isolated_model_class.split('.')[-1]

    import_cmd = 'from '+nmsp+' import '+cls+' as Model'+str(i)
    print(import_cmd)
    exec(import_cmd)
    
# Create work item list
work_items = []

for model in model_classes:
    work_items.append({"model_class": model.isolated_model_class })
        
def dowork(item):
    results = []
    
    import prev_ob_models
    exec('cell = '+ str(item["model_class"]) +'()')

    model = NeuronCellModel(cell.soma(0.5),name=cell.__class__.__module__+'.'+cell.__class__.__name__)


    for test in tests:
        try:
            print('Starting', item, test)
            result = test.generate_prediction(model)
            print('Finished', item, test, result)

        except:
            import traceback
            result = traceback.format_exc()

        results.append(result)

    return item["model_class"], results


creating specific test class: AfterHyperpolarizationAmplitudeTestYu2015
creating specific test class: AfterHyperpolarizationAmplitudeTestBurtonUrban2014
creating specific test class: AfterHyperpolarizationTimeTestYu2015
creating specific test class: AfterHyperpolarizationTimeTestBurtonUrban2014
creating specific test class: SpikeAmplitudeTestYu2015
creating specific test class: SpikeAmplitudeTestBurtonUrban2014
creating specific test class: SpikeHalfWidthTestYu2015
creating specific test class: SpikeHalfWidthTestBurtonUrban2014
creating specific test class: SpikePeakTestYu2015
creating specific test class: SpikeThresholdTestYu2015
creating specific test class: SpikeThresholdTestBurtonUrban2014
creating specific test class: CellCapacitanceTestYu2015
creating specific test class: CellCapacitanceTestBurtonUrban2014
creating specific test class: FISlopeTestBurtonUrban2014
creating specific test class: InputResistanceTestYu2015
creating specific test class: InputResistanceTestBurtonUrban201

In [3]:
processes = 8   
from multiprocessing import Pool
pool = Pool(processes = processes, maxtasksperchild=1)
result = pool.map(dowork, work_items)
pool.terminate()
pool.join()
result

	1 
	1001 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
	1 
	1 
	1 
	0 
	1 
	0 
	1 
	0 
('Starting', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.AfterHyperpolarizationAmplitudeTestYu2015 object at 0x7f452162b550>)
	0 
('Finished', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.AfterHyperpolarizationAmplitudeTestYu2015 object at 0x7f452162b550>, array([16.42546115]) * mV)
('Starting', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.AfterHyperpolarizationAmplitudeTestBurtonUrban2014 object at 0x7f4521614950>)
	0 
	0 
	0 
('Finished', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.AfterHyperpolarizationAmplitudeTestBurtonUrban2014 object at 0x7f4521614950>, array([17.2789341]) * mV)
	0 
('Starting', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.AfterHyperpolarizationTimeTestYu2015 object at 0x7f452162bdd0>)
	1 
('Finished', {'model_class': u'prev_ob_models.Chen20

<< glomerulus connected up >>
<< secondary dendrites all connected >>
('Starting', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.InputResistanceTestHovis2010 object at 0x7f45215f0550>)
	1 
('Finished', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.InputResistanceTestHovis2010 object at 0x7f45215f0550>, array(104.59925123) * megaohm)
	1 
	1 
	0 
('Starting', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.InputResistanceTestShpak2012 object at 0x7f45215f0910>)
	0 
	0 
	0 
	0 
	0 
	0 
	0 
<< Setting membrane properties and inserting channels >>
<< Electrode leak 120 megohms at soma >>
('Starting', {'model_class': u'prev_ob_models.BhallaBower1993.isolated_cells.MC'}, <__main__.AfterHyperpolarizationAmplitudeTestYu2015 object at 0x7f452162b550>)
('Finished', {'model_class': u'prev_ob_models.BhallaBower1993.isolated_cells.MC'}, <__main__.AfterHyperpolarizationAmplitudeTestYu2015 object at 0x7f452162b550>, array([

('Starting', {'model_class': u'prev_ob_models.BhallaBower1993.isolated_cells.MC'}, <__main__.ISICVTestBurtonUrban2014 object at 0x7f452162b810>)
('Finished', {'model_class': u'prev_ob_models.BhallaBower1993.isolated_cells.MC'}, <__main__.ISICVTestBurtonUrban2014 object at 0x7f452162b810>, array(0.01949758) * dimensionless)
('Starting', {'model_class': u'prev_ob_models.BhallaBower1993.isolated_cells.MC'}, <__main__.ReboundSpikingTestBurtonUrban2014 object at 0x7f45215dbc90>)
('Finished', {'model_class': u'prev_ob_models.BhallaBower1993.isolated_cells.MC'}, <__main__.ReboundSpikingTestBurtonUrban2014 object at 0x7f45215dbc90>, False)
('Starting', {'model_class': u'prev_ob_models.BhallaBower1993.isolated_cells.MC'}, <__main__.ReboundSpikingTestJohnsonDelaney2010 object at 0x7f4521591190>)
('Finished', {'model_class': u'prev_ob_models.Davison2000.isolated_cells.MC'}, <__main__.SpikeHalfWidthTestBurtonUrban2014 object at 0x7f452162b650>, array(0.5) * ms)
('Finished', {'model_class': u'prev_

('Starting', {'model_class': u'prev_ob_models.BhallaBower1993.isolated_cells.MC'}, <__main__.RheobaseTestBurtonUrban2014 object at 0x7f4521591790>)
	0 
('Finished', {'model_class': u'prev_ob_models.David2008.isolated_cells.MC'}, <__main__.AfterHyperpolarizationTimeTestBurtonUrban2014 object at 0x7f45215db090>, array(0.5) * ms)
	0 
('Starting', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.ReboundSpikingTestBurtonUrban2014 object at 0x7f45215dbc90>)
Code block 'MembraneTimeConstantTestYu2015' took: 80.1529884338 ms
('Starting', {'model_class': u'prev_ob_models.David2008.isolated_cells.MC'}, <__main__.SpikeAmplitudeTestYu2015 object at 0x7f45215db310>)
	0 
('Finished', {'model_class': u'prev_ob_models.McTavish2012.isolated_cells.MC'}, <__main__.SpikeAmplitudeTestYu2015 object at 0x7f45215db310>, array([91.91493378]) * mV)
('Finished', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.ReboundSpikingTestBurtonUrban2014 object at 0x7f45215db

Code block 'MembraneTimeConstantTestYu2015' took: 60.0640773773 ms
('Finished', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.SpikeAccommodationTimeConstantTestBurtonUrban2014 object at 0x7f452162bf10>, array(0.55329135) * ms)
('Starting', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.SpikeAccommodationTimeConstantTestZibman2011 object at 0x7f45215f0610>)
Code block 'CellCapacitanceTestYu2015' took: 97.0599651337 ms
('Finished', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.SpikeAccommodationTimeConstantTestZibman2011 object at 0x7f45215f0610>, array(0.71850081) * ms)
('Starting', {'model_class': u'prev_ob_models.Chen2002.isolated_cells.MC'}, <__main__.MembraneTimeConstantTestYu2015 object at 0x7f45215a62d0>)
('Finished', {'model_class': u'prev_ob_models.David2008.isolated_cells.MC'}, <__main__.CellCapacitanceTestYu2015 object at 0x7f45215f0090>, array(805.16262776) * pF)
('Finished', {'model_class': u'pre

...

Code block 'ISICVTestBurtonUrban2014' took: 281554.538012 ms
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC1'}, <__main__.ISICVTestBurtonUrban2014 object at 0x7f452162b810>, array(nan) * dimensionless)
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC1'}, <__main__.ReboundSpikingTestBurtonUrban2014 object at 0x7f45215dbc90>)
Code block 'ReboundSpikingTestBurtonUrban2014' took: 4605.79609871 ms
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC1'}, <__main__.ReboundSpikingTestBurtonUrban2014 object at 0x7f45215dbc90>, False)
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC1'}, <__main__.ReboundSpikingTestJohnsonDelaney2010 object at 0x7f4521591190>)
Code block 'ReboundSpikingTestJohnsonDelaney2010' took: 5109.26890373 ms
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC1'}, <__main__.ReboundSpikingTestJohnsonDelaney2010 object at 0x7f4521591190>, False)
('

('Starting', {'model_class': u'prev_ob_models.Short2016.isolated_cells.MC'}, <__main__.MembraneTimeConstantTestHovis2010 object at 0x7f45215a66d0>)
Code block 'MembraneTimeConstantTestHovis2010' took: 2.25496292114 ms
('Finished', {'model_class': u'prev_ob_models.Short2016.isolated_cells.MC'}, <__main__.MembraneTimeConstantTestHovis2010 object at 0x7f45215a66d0>, array(11.) * ms)
('Starting', {'model_class': u'prev_ob_models.Short2016.isolated_cells.MC'}, <__main__.MembraneTimeConstantTestZibman2011 object at 0x7f45215db7d0>)
Code block 'MembraneTimeConstantTestZibman2011' took: 3.62181663513 ms
('Finished', {'model_class': u'prev_ob_models.Short2016.isolated_cells.MC'}, <__main__.MembraneTimeConstantTestZibman2011 object at 0x7f45215db7d0>, array(11.) * ms)
Code block 'PassivePropertyTestHelperHu2016' took: 4443.24207306 ms
Code block 'SagVoltageTestHu2016' took: 4475.28290749 ms
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC1'}, <__main__.SagVoltageTestHu

Code block 'ReboundSpikingTestJohnsonDelaney2010' took: 5097.98312187 ms
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC6'}, <__main__.ReboundSpikingTestJohnsonDelaney2010 object at 0x7f4521591190>, False)
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC6'}, <__main__.RestingVoltageTestYu2015 object at 0x7f4521591410>)
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC6'}, <__main__.RestingVoltageTestYu2015 object at 0x7f4521591410>, array(-67.35197911) * mV)
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC6'}, <__main__.RestingVoltageTestBurtonUrban2014 object at 0x7f452162b690>)
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC6'}, <__main__.RestingVoltageTestBurtonUrban2014 object at 0x7f452162b690>, array(-67.35243976) * mV)
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC6'}, <__main__.RestingVoltageTestAbraham2010 object a

('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC6'}, <__main__.SpikeAccommodationTimeConstantTestZibman2011 object at 0x7f45215f0610>, array(500.) * ms)
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC6'}, <__main__.MembraneTimeConstantTestYu2015 object at 0x7f45215a62d0>)
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC6'}, <__main__.MembraneTimeConstantTestYu2015 object at 0x7f45215a62d0>, array(4.) * ms)
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC6'}, <__main__.MembraneTimeConstantTestBurtonUrban2014 object at 0x7f45215a6050>)
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC6'}, <__main__.MembraneTimeConstantTestBurtonUrban2014 object at 0x7f45215a6050>, array(4.) * ms)
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC6'}, <__main__.MembraneTimeConstantTestHovis2010 object at 0x7f45215a66d0>)
Code block 'MembraneTimeCon

[(u'prev_ob_models.BhallaBower1993.isolated_cells.MC',
  [array([17.35223496]) * mV,
   array([17.64018939]) * mV,
   array(2.5) * ms,
   array(0.75) * ms,
   array([66.21673956]) * mV,
   array([63.20048465]) * mV,
   array(0.5) * ms,
   array(0.5) * ms,
   array(18.81658487) * mV,
   array([-47.40015469]) * mV,
   array([-47.11369697]) * mV,
   array(644.69948933) * pF,
   array(644.77643822) * pF,
   array(100.) * Hz/nA,
   array(55.83996978) * megaohm,
   array(55.83330573) * megaohm,
   array(54.29425337) * megaohm,
   array(54.29425337) * megaohm,
   array(54.29425337) * megaohm,
   array(54.45248522) * megaohm,
   array(54.29425337) * megaohm,
   array(54.29425337) * megaohm,
   array(0.01404365) * dimensionless,
   array(0.01949758) * dimensionless,
   False,
   False,
   array(-65.839002) * mV,
   array(-65.839002) * mV,
   array(-65.839002) * mV,
   array(-65.839002) * mV,
   array(213.04025291) * pA,
   array([0.2135554]) * mV,
   array([0.2134225]) * mV,
   array(0.21340206

In [17]:
df = []
for model in result:
    row = {"model": model[0]}
    for t, test in enumerate(tests):
        row[test.__class__.__name__] = model[1][t]
    df.append(row)
        
df = DataFrame(df)
df


,AfterHyperpolarizationAmplitudeTestBurtonUrban2014,AfterHyperpolarizationAmplitudeTestYu2015,AfterHyperpolarizationTimeTestBurtonUrban2014,AfterHyperpolarizationTimeTestYu2015,CellCapacitanceTestBurtonUrban2014,CellCapacitanceTestYu2015,FISlopeTestBurtonUrban2014,ISICVTestBurtonUrban2014,ISICVTestYu2015,InputResistanceTestAbraham2010,...,SpikeAccommodationTimeConstantTestBurtonUrban2014,SpikeAccommodationTimeConstantTestZibman2011,SpikeAmplitudeTestBurtonUrban2014,SpikeAmplitudeTestYu2015,SpikeHalfWidthTestBurtonUrban2014,SpikeHalfWidthTestYu2015,SpikePeakTestYu2015,SpikeThresholdTestBurtonUrban2014,SpikeThresholdTestYu2015,model
0,[17.640189386524575],[17.352234959382415],0.7500000000000284,2.4999999999999467,644.7764382161346,644.6994893313141,100.0,0.01949758423845359,0.0140436450051258,54.29425336529173,...,169.91234687611046,500.0,[63.20048464624422],[66.21673955705481],0.5,0.5,18.816584865073498,[-47.11369696790417],[-47.400154691981314],prev_ob_models.BhallaBower1993.isolated_cells.MC
1,[17.278934096595748],[16.425461152119603],1.2499999999999734,2.4999999999999467,178.1081906747358,177.95987364400685,440.0,0.002725407835316132,0.0037076704194513475,104.59925122552039,...,0.5532913502367369,0.71850080639508,[69.83239120588837],[73.50647273653684],0.75,0.75,17.223754476684974,[-55.42737603966431],[-56.282718259851855],prev_ob_models.Chen2002.isolated_cells.MC
2,[23.57213947764062],[22.653907903094378],0.500000000000167,2.2500000000000853,802.398061727078,805.1626277627338,80.0,0.017174730726524572,0.013780146383435022,86.01979291481196,...,146.0626953327339,500.0,[69.89153412432849],[63.59052968790094],0.5,0.25,20.366953437342058,[-42.2941320323135],[-43.22357625055888],prev_ob_models.David2008.isolated_cells.MC
3,[23.153320939907566],[22.47144638487631],0.7500000000000284,2.2500000000000853,829.3981527471584,831.4614755775153,80.0,0.015957376674673302,0.012476998570883585,85.49686448207555,...,144.8644644499656,500.0,[54.582886272708535],[65.10393749638587],0.5,0.25,21.633039835222466,[-42.77794521878612],[-43.4708976611634],prev_ob_models.Davison2000.isolated_cells.MC
4,[32.374446872025544],[1.9718316924567603],0.7500000000000284,1.7499999999999183,99.96789024755688,99.72090068382921,10.0,"Traceback (most recent call last):\n File ""<i...","Traceback (most recent call last):\n File ""<i...",223.0797308018118,...,"Traceback (most recent call last):\n File ""<i...",500.0,[34.85426982856693],[66.03873608028383],0.5,0.75,1.0385076940929427,[-34.83896234716599],[-65.00022838619088],prev_ob_models.Djurisic2008.isolated_cells.MC
5,[24.460686535326374],[22.14883336954246],0.7500000000000284,2.4999999999999467,832.851550034309,835.6499401294801,80.0,0.015816400869572082,0.012336653522572572,85.19049774747108,...,149.89819415929446,500.0,[60.65829167256379],[77.52927684424975],0.25,0.25,33.74777111224112,[-41.46761045630858],[-43.78150573200864],prev_ob_models.KaplanLansner2014.isolated_cell...
6,[35.86533950066723],[33.5776638814635],0.7500000000000284,3.249999999999975,110.20514298418449,110.95937944806708,50.0,2.1020448060313752,1.7935428784818856,188.436955358601,...,0.3827798677545078,500.0,[64.44432748147918],[69.38989757194045],0.5,0.5,26.554382328603268,[-40.496594352814554],[-42.83551524333718],prev_ob_models.LiCleland2013.isolated_cells.MC
7,[13.47238863210169],[15.515795423950884],2.75000000000003,5.499999999999838,179.04804022387336,178.91356200871058,310.0,0.009724262341926498,0.01135221828589726,62.534148792938254,...,21.077586969581347,500.0,[91.3112167235961],[91.914933784457],1.75,1.75,39.230370988640566,[-52.34401938276198],[-52.68456279581644],prev_ob_models.McTavish2012.isolated_cells.MC
8,[17.239337194790885],[18.402378902286515],2.4999999999999467,5.249999999999977,117.41951636248496,117.32983970834232,370.0,0.009211402584963087,0.011143254139380268,97.50066268021185,...,23.925343076629396,500.0,[89.17445845628981],[91.57581455791978],1.75,1.75,41.46685421181244,[-48.87143563965744],[-50.10896034610734],pr

In [4]:
raise Exception()

Exception: 

In [22]:
work_items[15]

{'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}

In [23]:
for item in work_items[15:16]:
    dowork(item)

('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}, <__main__.AfterHyperpolarizationAmplitudeTestYu2015 object at 0x7f452162b550>)
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}, <__main__.AfterHyperpolarizationAmplitudeTestYu2015 object at 0x7f452162b550>, array([1.07284496]) * mV)
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}, <__main__.AfterHyperpolarizationAmplitudeTestBurtonUrban2014 object at 0x7f4521614950>)
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}, <__main__.AfterHyperpolarizationAmplitudeTestBurtonUrban2014 object at 0x7f4521614950>, array([0.]) * mV)
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}, <__main__.AfterHyperpolarizationTimeTestYu2015 object at 0x7f452162bdd0>)
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}, <__main__.AfterHyperpolarizationTimeTestYu2015 object at 0x7f452162b

/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py:594: RuntimeWarning: divide by zero encountered in divide
  current_targetHz = rheobase + (target_freq - freq_rb) / freq_slope


Code block 'ISICVTestYu2015' took: 73847.9399681 ms
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}, <__main__.ISICVTestBurtonUrban2014 object at 0x7f452162b810>)
Code block 'ISICVTestBurtonUrban2014' took: 88717.664957 ms
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}, <__main__.ReboundSpikingTestBurtonUrban2014 object at 0x7f45215dbc90>)
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}, <__main__.ReboundSpikingTestBurtonUrban2014 object at 0x7f45215dbc90>, False)
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}, <__main__.ReboundSpikingTestJohnsonDelaney2010 object at 0x7f4521591190>)
('Finished', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}, <__main__.ReboundSpikingTestJohnsonDelaney2010 object at 0x7f4521591190>, False)
('Starting', {'model_class': u'prev_ob_models.Oconnor2012.isolated_cells.MC2'}, <__main__.RestingVoltageTestYu2015 object 

In [ ]:
#cache.clear()
import prev_ob_models
cell = prev_ob_models.MiglioreMcTavish2013.isolated_cells.MC()
model = NeuronCellModel(cell.soma(0.5),name=cell.__class__.__module__+'.'+cell.__class__.__name__)
SpikeAmplitudeTestYu2015().generate_prediction(model)

In [ ]:
model = None
cell = None
raise Exception()

In [ ]:
for i, test in enumerate(tests):
    #print(i, 'judging', test)
    print(test.generate_prediction(model))
    

In [ ]:
raise Exception()

In [ ]:
test.observation

In [ ]:
with CodeTimer():
    TestSuite(tests).judge(model)

In [ ]:
from neuronunit.tests.olfactory_bulb.utilities import cache
#cache.clear()

In [ ]:

class Test(BurtonUrban2015, SpikeThresholdTest):
    pass

test = Test().generate_prediction(model)

In [ ]:
test

In [ ]:
#rb = RheobaseTestBurtonUrban2014().generate_prediction(model)
rb = 0.02438785 * pq.nA

In [ ]:
SpikeAccommodationTimeConstantTestZibman2011(rb).generate_prediction(model)

In [ ]:
SpikeAccommodationTestBurtonUrban2014(rb).generate_prediction(model)

In [ ]:
ISICVTestBurtonUrban2014(rb).generate_prediction(model)

In [ ]:
rb = RheobaseTestYu2015().generate_prediction(model)

In [ ]:
ISICVTestYu2015(rb).generate_prediction(model)

In [ ]:
0/0

In [ ]:
ReboundSpikingTestBurtonUrban2014().generate_prediction(model)

In [ ]:
ReboundSpikingTestJohnsonDelaney2010().generate_prediction(model)

In [ ]:
rb = RheobaseTestBurtonUrban2014().generate_prediction(model)
rb

In [ ]:
voltage = model.inject_square_current({"delay": 1000*pq.ms,"duration": 2000*pq.ms, "amplitude": rb})

In [ ]:
plt.plot(voltage)

In [ ]:
voltage.magnitude[np.where(voltage.times > 1000*pq.ms)]

In [ ]:
rb = 0.02438782 * pq.nA

In [ ]:
# %matplotlib notebook
# %matplotlib notebook
# %matplotlib notebook
# %matplotlib notebook

In [ ]:
rb = RheobaseTestYu2015().generate_prediction(model)
rb

In [ ]:
SagVoltageTestAngelo2012().generate_prediction(model)
SagVoltageTestBurtonUrban2014().generate_prediction(model)
SagVoltageTestYu2015().generate_prediction(model)
SagVoltageTestHu2016().generate_prediction(model)

In [ ]:
AfterHyperpolarizationAmplitudeTestYu2015(rb).generate_prediction(model)

In [ ]:
AfterHyperpolarizationTimeTestYu2015(rb).generate_prediction(model)

In [ ]:
AfterHyperpolarizationAmplitudeTestBurtonUrban2014(rb).generate_prediction(model)

In [ ]:
AfterHyperpolarizationTimeTestBurtonUrban2014(rb).generate_prediction(model)

In [ ]:
1/0

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

v = aps[0]
plt.plot(v["voltage"])
#plt.plot(v3_z[0:get_zero_crossings(v3_z)[0]+1])
#plt.xlim((0,80))
plt.show()

In [ ]:
test1 = SagVoltageTestAngelo2012(observation={"mean":0*pq.mV, 'std':1*pq.mV,'n': 10})
test2 = SagVoltageTestBurtonUrban2014(observation={"mean":0*pq.mV, 'std':1*pq.mV,'n': 10})
test3 = SagVoltageTestYu2015(observation={"mean":0*pq.mV, 'std':1*pq.mV,'n': 10})
test4 = SagVoltageTestHu2016(observation={"mean":0*pq.mV, 'std':1*pq.mV,'n': 10})

#suite = test1 
suite = TestSuite(tests=[test1,test2,test3,test4])

In [ ]:
suite.judge(model)